In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d

import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib inline

import copy
import json
from tqdm import tqdm

import pymultinest
import petitRADTRANS.nat_cst as nc

import retrieval_base.auxiliary_functions as af

def read_results(prefix, n_params, w_set='K2166'):

    # Set-up analyzer object
    analyzer = pymultinest.Analyzer(
        n_params=n_params, 
        outputfiles_basename=prefix
        )
    stats = analyzer.get_stats()

    # Load the equally-weighted posterior distribution
    posterior = analyzer.get_equal_weighted_posterior()
    posterior = posterior[:,:-1]

    # Read the parameters of the best-fitting model
    bestfit = np.array(stats['modes'][0]['maximum a posterior'])

    PT = af.pickle_load(prefix+'data/bestfit_PT.pkl')
    Chem = af.pickle_load(prefix+'data/bestfit_Chem.pkl')

    m_spec = af.pickle_load(prefix+f'data/bestfit_m_spec_{w_set}.pkl')
    d_spec = af.pickle_load(prefix+f'data/d_spec_{w_set}.pkl')

    LogLike = af.pickle_load(prefix+f'data/bestfit_LogLike_{w_set}.pkl')

    try:
        Cov = af.pickle_load(prefix+f'data/bestfit_Cov_{w_set}.pkl')
    except:
        Cov = None

    int_contr_em           = np.load(prefix+f'data/bestfit_int_contr_em_{w_set}.npy')
    int_contr_em_per_order = np.load(prefix+f'data/bestfit_int_contr_em_per_order_{w_set}.npy')
    int_opa_cloud          = np.load(prefix+f'data/bestfit_int_opa_cloud_{w_set}.npy')

    f = open(prefix+'data/bestfit.json')
    bestfit_params = json.load(f)
    f.close()

    return posterior, bestfit, PT, Chem, int_contr_em, int_contr_em_per_order, int_opa_cloud, m_spec, d_spec, LogLike, Cov, bestfit_params

In [ ]:
prefix_1 = './retrieval_outputs/<...>/test_'
w_set = 'K2166'
n_params = 24

res = read_results(prefix=prefix_1, n_params=n_params, w_set=w_set)
posterior_1, bestfit_1, PT_1, Chem_1, _, _, _, m_spec_1, d_spec_1, LogLike_1, Cov_1, bestfit_params_1 = res
del res

In [ ]:
def get_pRT_atm(prefix, w_set, bestfit_params, Chem, PT, vsini_local=0, line_species=None):

    # Load the pRT-model object
    pRT_atm = af.pickle_load(f'{prefix}data/pRT_atm_broad_{w_set}.pkl')

    # Compute global model spectrum
    params_global = copy.deepcopy(bestfit_params['params'])
    pRT_atm.params = params_global

    m_spec_global = pRT_atm(
        Chem.mass_fractions, PT.temperature, params_global, 
        get_contr=False, get_full_spectrum=True
        )
    m_spec_global.wave_pRT_grid = copy.copy(pRT_atm.wave_pRT_grid)
    m_spec_global.flux_pRT_grid = copy.copy(pRT_atm.flux_pRT_grid)

    # Compute local model spectrum
    params_local = copy.deepcopy(params_global)
    params_local['vsini'] = vsini_local
    pRT_atm.params = params_local

    # Local abundances
    mf_local = Chem.mass_fractions.copy()
    if line_species is not None:
        for key_i in mf_local.keys():
            # Set abundances of other species to 0
            if key_i != line_species:
                mf_local[key_i] *= 0

    m_spec_local = pRT_atm(
        mf_local, PT.temperature, params_local, 
        get_contr=False, get_full_spectrum=True
        )
    m_spec_local.wave_pRT_grid = copy.copy(pRT_atm.wave_pRT_grid)
    m_spec_local.flux_pRT_grid = copy.copy(pRT_atm.flux_pRT_grid)

    return m_spec_global, m_spec_local

m_spec_global_1, m_spec_local_1, Rot_1 = get_pRT_atm(
    prefix_1, w_set, bestfit_params_1, copy.deepcopy(Chem_1), PT_1
    )

In [ ]:
def get_CCF(
        d_spec, m_spec_local, m_spec_global, LogLike, Cov, rv=np.arange(-400,400+1e-6,1), 
        subtract_global_from_template=True, filter_width=100, return_CCF_on_data=True
        ):
    
    # Store correlation-coefficient for each (rv, order, detector)
    CCF_on_res = np.nan * np.ones(
        (len(rv), d_spec.n_orders, d_spec.n_dets)
        )
    
    # Get the residual spectrum
    d_wave = np.copy(d_spec.wave)
    d_res  = np.copy(d_spec.flux) - LogLike.f[:,:,None]*m_spec_global.flux
    
    # Remove any broad features in the residuals with high-pass filter
    mask = d_spec.mask_isfinite

    lp_d_res = np.nan*np.ones_like(d_res)
    lp_d_res[mask] = gaussian_filter1d(
        d_res[mask], sigma=filter_width, axis=-1, mode='nearest'
        )
    d_res -= lp_d_res
    del lp_d_res

    if return_CCF_on_data:

        # Also compute cross-correlation for original flux
        CCF_on_data = np.nan * np.ones(
            (len(rv), d_spec.n_orders, d_spec.n_dets)
            )
        
        # Apply high-pass filter to the observed spectrum
        d_flux = np.copy(d_spec.flux)
        lp_d_flux = np.nan*np.ones_like(d_flux)
        lp_d_flux[mask] = gaussian_filter1d(
            d_flux[mask], sigma=filter_width, axis=-1, mode='nearest'
            )
        d_flux -= lp_d_flux
        del lp_d_flux

    # Loop over all (RVs, orders, detectors)
    for i, rv_i in enumerate(tqdm(rv)):

        for j in range(d_spec.n_orders):
            
            # Shift the model spectrum
            m_wave_local = np.copy(m_spec_local.wave_pRT_grid[j])
            m_wave_local *= (1 + rv_i/(nc.c*1e-5))

            m_flux_local = np.copy(m_spec_local.flux_pRT_grid[j])

            for k in range(d_spec.n_dets):

                # Subtract the global model?
                if subtract_global_from_template:
                    m_res_local_jk = LogLike.f[j,k] * (m_flux_local - m_spec_global.flux_pRT_grid[j])
                else:
                    m_res_local_jk = LogLike.f[j,k] * m_flux_local
                
                # Optimal scaling
                #m_res_local *= LogLike.f[j,:][:,None]
                
                # Don't consider the nans
                mask_jk = d_spec.mask_isfinite[j,k]

                if not mask_jk.any():
                    continue

                # Interpolate onto the data's wavelength grid
                m_res_local_jk = np.interp(
                    d_wave[j,k], xp=m_wave_local, fp=m_res_local_jk
                    )
                
                lp_m_res_local_jk = np.nan*np.ones_like(m_res_local_jk)
                lp_m_res_local_jk[mask_jk] = gaussian_filter1d(
                    m_res_local_jk[mask_jk], sigma=filter_width, axis=-1, mode='nearest'
                    )
                m_res_local_jk -= lp_m_res_local_jk
                
                # Compute the covariance-weighted dot-product
                CCF_on_res[i,j,k] = np.dot(
                    m_res_local_jk[mask_jk], 1/LogLike.beta[j,k]**2 * Cov[j,k].solve(d_res[j,k,mask_jk])
                    )
                
                if return_CCF_on_data:
                    CCF_on_data[i,j,k] = np.dot(
                        m_res_local_jk[mask_jk], 1/LogLike.beta[j,k]**2 * Cov[j,k].solve(d_flux[j,k,mask_jk])
                        )

    if return_CCF_on_data:
        return CCF_on_res, CCF_on_data, rv
    else:
        return CCF_on_res, rv

CCF_on_res_1, CCF_on_data_1, rv = get_CCF(
    d_spec_1, m_spec_local_1, m_spec_global_1, LogLike_1, Cov_1, 
    )

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))

# Sum over the (order, detector)-axes
ax.plot(rv, np.nansum(CCF_on_res_1, axis=(1,2)), c='k')

# Show the vsini-range
vsini = bestfit_params_1['params']['vsini']
ax.axvline(-vsini, color='k', alpha=0.5, ls=(0,(5,5)))
ax.axvline(+vsini, color='k', alpha=0.5, ls=(0,(5,5)))

ax.set(xlim=(rv.min(), rv.max()))
plt.show()